<a href="https://colab.research.google.com/github/Rudrajoshiqyay/htmlrepo/blob/main/interface_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Install packages
!pip install yfinance plotly --quiet



In [5]:
# Step 2: Import libraries and get data
import yfinance as yf
import plotly.graph_objects as go

# Replace with your preferred stock symbol
ticker = "RELIANCE.NS"
data = yf.download(ticker, start="2024-01-01", end="2024-03-31", interval="1d")

# Define your entry and exit levels manually (or compute from strategy)
entry_level = 2400
exit_level = 2550


[*********************100%***********************]  1 of 1 completed


In [10]:
!pip install plotly pandas

import pandas as pd
import plotly.graph_objects as go

# Load CSV
df = pd.read_csv("/content/Quote-Equity-RELIANCE-EQ-09-03-2025-to-09-04-2025.csv")
df.columns = df.columns.str.strip()

# Clean data
for col in ['OPEN', 'HIGH', 'LOW', 'close']:
    df[col] = df[col].str.replace(',', '').astype(float)

df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
df = df.sort_values('Date')

# Detect candlestick patterns
def detect_patterns(row):
    body = abs(row['OPEN'] - row['close'])
    range_ = row['HIGH'] - row['LOW']
    upper_shadow = row['HIGH'] - max(row['OPEN'], row['close'])
    lower_shadow = min(row['OPEN'], row['close']) - row['LOW']

    if body < range_ * 0.1 and upper_shadow > lower_shadow * 2:
        return 'Inverted Hammer'
    elif body < range_ * 0.1 and lower_shadow > upper_shadow * 2:
        return 'Hammer'
    elif body < range_ * 0.05:
        return 'Doji'
    else:
        return ''

df['Pattern'] = df.apply(detect_patterns, axis=1)

# Plot candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=df['Date'],
    open=df['OPEN'],
    high=df['HIGH'],
    low=df['LOW'],
    close=df['close'],
    name="Price",
    increasing_line_color='green',
    decreasing_line_color='red'
)])

# Add Entry/Exit lines
entry_level = 1150
exit_level = 1200
fig.add_hline(y=entry_level, line_dash="dot", line_color="blue", annotation_text="Entry", annotation_position="top left")
fig.add_hline(y=exit_level, line_dash="dot", line_color="orange", annotation_text="Exit", annotation_position="top left")

# Annotate patterns
for i, row in df.iterrows():
    if row['Pattern']:
        fig.add_annotation(
            x=row['Date'],
            y=row['HIGH'],
            text=row['Pattern'],
            showarrow=True,
            arrowhead=1,
            ax=0,
            ay=-40,
            bgcolor="lightyellow"
        )

fig.update_layout(
    title="RELIANCE Candlestick Chart with Entry/Exit & Patterns",
    xaxis_title="Date",
    yaxis_title="Price (INR)",
    xaxis_rangeslider_visible=False,
    template="plotly_white",
    height=600
)

fig.show()


In [11]:
fig.write_html("candlestick_chart.html")
